In [1]:
import pandas as pd
from Bio import Entrez
import xml.etree.ElementTree as ET


# Path to XML Database
  1) search keyword (ex. "Leptospira") in BioSample db of NCBI
  2) Download XML file

In [8]:
path="/Users/rachel/Dropbox/5.Rachel-projects/Phylogeography/Interrogans/" # Interrogans Path
all_lepto_path="/Users/rachel/Dropbox/5.Rachel-projects/Phylogeography/SeroAll/" # all pathogenic leptopsira path (with sero)

In [9]:
# metadata for all 389 Leptopsira with assembly dates

metadata_db = ET.parse(all_lepto_path + "ncbi_biosample_db_all.xml") # search "Leptospira interrogan" in NCBI database

# write xml into a dataframe

with open(all_lepto_path + "ncbi_biosample_db_all.csv","w") as f:
    headers="BioSample.Accession,Bioproject,Collection.Date,La.lon,Host.Name,Geographic.Location,country,Isolation.Source,Strain,Serovar,Genome.Name,Institute\n"
    f.write(headers)

all_biosample_root= metadata_db.getroot()

for biosample in all_biosample_root.findall("BioSample"):
    # set all variable to empty str to avoid nonotype
    bios_acc=""
    collection_date=""
    lat_lon=""
    host=""
    location=""
    iso_source =""
    strain =""
    serovar =""
    institute=""
    country=""
    bioproject=""
    
    bios_acc = biosample.find("Ids").find("Id").text
    name = biosample.find("Description").find("Title").text
    institute = biosample.find("Owner").find("Name").text
    if institute == None:
        institute = ""
    elif "," in institute:
        institute = institute.replace(",",":")
    
    Link= biosample.find("Links")
    if Link != None:
        for link in Link.findall("Link"):
            if link.attrib.get("target") == "bioproject":
                bioproject=link.attrib.get("label")
                
    
    for attribute in biosample.find("Attributes").findall("Attribute"):
        if attribute.attrib.get("display_name") == "collection date":
            collection_date = attribute.text
        if attribute.attrib.get("attribute_name") == "lat_lon":
            lat_lon = attribute.text
            if "," in lat_lon:
                lat_lon= lat_lon.replace(","," ")
        if attribute.attrib.get("display_name") == "host":
            host=attribute.text
        if attribute.attrib.get("display_name") == "geographic location":
            location = attribute.text
            if "," in location:
                location= location.replace(","," ")
            if ":" in location:
                country = location.split(":")[0] # extract country
            else:
                country=location
        if attribute.attrib.get("display_name") == "isolation source":
            iso_source = attribute.text
            if "," in iso_source:
                iso_source= iso_source.replace(","," ")
        if attribute.attrib.get("display_name") == "strain":
            strain = attribute.text
        if attribute.attrib.get("display_name") == "serovar":
            serovar = attribute.text

    complete_str =  bios_acc+ ","+ bioproject+","  +collection_date + "," + lat_lon + "," +  host+ ","+ location+","+country+ ","+iso_source+ ","+strain+ ","+serovar+ ","+ name+ ","+institute + "\n"
   
    with open(all_lepto_path + "ncbi_biosample_db_all.csv", "a+") as f:
        f.write(complete_str) 
    

        
    

# Merge interrogans isolates with their metadata extracted from XML db above

In [4]:
# # read 440 Leptospira isolates biosample list, coverage and other info 
# interrogans_440= pd.read_csv(path+"all_interrogans_440.csv", index_col=False)
# # rename the acc column to match the db, prepare for merging
# interrogans_440=interrogans_440.rename(columns = {"Sample":"BioSample.Accession"}) 
# # read the database we created from last block
# interrogan_db=pd.read_csv(path+"ncbi_biosample_db_all.csv")
# merge all Leptospira isolates (440) with data frame created above, drop those not in the interrogans list
# interrogans_440_2 =interrogans_440.merge(interrogan_db,  how="left",on="BioSample.Accession")
# # interrogans_440_2.to_csv(path+"all_interrogans_metadata_440.csv")

ParserError: Error tokenizing data. C error: Expected 12 fields in line 385, saw 13


# Merge all Pathogenic Leptopsira isolate with their corresponding Metdata in the database


In [20]:
# read in the database
all_Lepto_db = pd.read_csv(all_lepto_path + "ncbi_biosample_db_all.csv")

# read in the pathogenic Leptospira isolates NCBI downloaded and self assembled and change column name
pathogenic_lepto = pd.read_csv(all_lepto_path + "ncbiself_assembled_pathogenic_782.csv").rename(columns={"Sample" : "BioSample.Accession"})


# merge two dataframe
pathogenic_lepto_metadata = pathogenic_lepto.merge(all_Lepto_db)

# write into file
pathogenic_lepto_metadata.to_csv(all_lepto_path + "ncbiself_metadata_pathogenic_782.csv")